In [1]:
python_libraries = ["numpy","pandas","gym","seaborn","pyvirtualdisplay","imageio","nnfigs","box2d-py", "gym pyvirtualdisplay", "setuptools", "ez_setup"]

packages = ["xvfb","x11-utils", "xvfb python-opengl ffmpeg", "cmake"]

with open('script.sh', 'w') as file:
  for library in python_libraries:
    file.write("pip install " + library +"\n")
  for package in packages:
    file.write("apt install " + package +"\n")

!bash script.sh

Reading package lists... Done
Building dependency tree       
Reading state information... Done
xvfb is already the newest version (2:1.19.6-1ubuntu4.10).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
x11-utils is already the newest version (7.7+3build1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-opengl is already the newest version (3.1.0+dfsg-1).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
xvfb is already the newest version (2:1.19.6-1ubuntu4.10).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [2]:
import gym
import numpy as np
import torch
from gym.wrappers import Monitor
import glob
import io
import base64
import time
import copy
from IPython.display import HTML
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay
from collections import OrderedDict
from google.colab import files

PATH = 'drive/MyDrive/'
# If you use Google Colab, use Google Drive to save and load files, else comment these ligns and use a local path
from google.colab import drive
drive.mount('/content/drive')

display = Display(visible=0, size=(1400, 900))
display.start()

"""
Utility functions to enable video recording of gym environment 
and displaying it.
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
in_dim = 24
out_dim = 4
h_dim = 16

class SingleAgent():
    
    def __init__(self):
        
        self.model = torch.nn.Sequential(
            torch.nn.Linear(in_dim, h_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(h_dim, out_dim),
            torch.nn.Sigmoid()
        )

        self.fitness = 0.0

        # List representation of the model weights and biases
        self.weights = self.from_state_dict_to_list()

    
    def act(self, input):
        actions = self.model.forward(input)
        return actions


    def update_weights(self):
        self.model.load_state_dict(self.from_list_to_state_dict())

    def from_list_to_state_dict(self):
        shapes = [self.model.state_dict()[x].shape for x in self.model.state_dict()]
        shapes_prod = [torch.tensor(s).numpy().prod() for s in shapes]

        partial_split = self.weights.split(shapes_prod)
        model_values = []
        for i in range(len(shapes)):
            model_values.append(partial_split[i].view(shapes[i]))

        state_dict = OrderedDict((key, value) for (key, value) in zip(self.model.state_dict().keys(), model_values))
        return state_dict

    def from_state_dict_to_list(self):
        parameters = self.model.state_dict().values()
        parameters = [x.flatten() for x in parameters]
        parameters = torch.cat(parameters, 0)
        return parameters

def get_fitness(agent):
    return agent.fitness

def run_single(env, agent, episode_length = 1000, render=False):
    observation = env.reset()
    fitness = 0

    while True:

        if render:
            env.render()
        
        input = torch.from_numpy(observation).float()
        action = agent.act(input).detach().numpy()
            
        observation, reward, done, info = env.step(action) 
        fitness += reward

        if done:
            break
    weights = agent.from_state_dict_to_list()
    return fitness, weights


def selection(old_gen):
    # Select individuals as parents for the next generation with an exponentially decreasing probability according to their fitness
    old_gen.sort(key=get_fitness, reverse=True)
    distr = np.array([np.exp(-i) for i in range(len(old_gen))])
    distr /= distr.sum()
    parent1, parent2 = np.random.choice(old_gen, 2, p=distr)
    return parent1, parent2

def mutation(child, p=0.1):
    # Mutate a random weight with probability p (follow a gaussian distribution centered on the mean of the previous weights)
        new_weights = child.weights.clone()
        n = torch.distributions.Normal(new_weights.mean(), new_weights.std())
        for i in range(child.weights.shape[0]):
            if np.random.rand() < p:
                new_weights[i] = n.sample() + np.random.randint(-20, 20)
        child.weights = new_weights
        child.update_weights()

def crossover(parent1, parent2):
    pos = np.random.randint(0, len(parent1.weights))
    child1, child2 = copy.deepcopy(parent1), copy.deepcopy(parent2)

    tmp = child1.weights[:pos].clone()
    child1.weights[:pos] = child2.weights[:pos]
    child2.weights[:pos] = tmp
    child1.update_weights()
    child2.update_weights()

    return child1, child2

def generation(old_gen):
    # Create a new generation using selection, crossover and mutation
    new_gen = [None for _ in range(nb_pop)]
    parent1, parent2 = selection(old_gen)

    for i in range(0, nb_pop-1, 2):
        child1, child2 = crossover(parent1, parent2)
        mutation(child1)
        mutation (child2)

        child1.fitness, child1.weights = run_single(env, child1)
        child2.fitness, child2.weights = run_single(env, child2)
 
        new_gen[i] = child1
        new_gen[i+1] = child2
    return new_gen

In [ ]:
nb_pop = 100
nb_generations = 1000

env = wrap_env(gym.make("BipedalWalker-v3"))
env._max_episode_steps = 100

old_gen = [SingleAgent() for _ in range(nb_pop)]
new_gen = [None for _ in range(nb_pop)]
stats = []

for gen in range(nb_generations):
    for i in range(nb_pop):
        agent = old_gen[i]
        agent.fitness, agent.weights = run_single(env, agent)
        

    new_gen = generation(old_gen)
    # Sort new gen by fitness, old gen is already sorted in generation function
    new_gen.sort(key=get_fitness, reverse=True)
    best_agent = old_gen[0]
    torch.save(best_agent.model.state_dict(), PATH+"genetic.pt")

    # Save stats about the current gen
    (max, mean, min) = (best_agent.fitness, np.mean(np.array([agent.fitness for agent in old_gen])), old_gen[-1].fitness)
    print(f"Generation {gen}: max {max} mean {mean} min{min}")
    stats.append((max, mean, min))
    with open(PATH+'stats.txt', 'a') as file:
        file.write(f"Generation {gen}: max {max} mean {mean} min{min} \n")
    
    # The next generation is composed of the best parents and the best children
    old_gen = old_gen[0:nb_pop//2] + new_gen[0:nb_pop//2]

    # if the mean fitness is larger than 300 we consider the problem solved
    if mean>300:
        break

env.close()
show_video()

In [3]:
# Here is a pretrained model, stored in the file genetic.pt
best_agent = SingleAgent()
best_agent.model.load_state_dict(torch.load(PATH+"genetic.pt"))
best_agent.model.eval()

Sequential(
  (0): Linear(in_features=24, out_features=16, bias=True)
  (1): ReLU()
  (2): Linear(in_features=16, out_features=4, bias=True)
  (3): Sigmoid()
)

In [4]:
env = wrap_env(gym.make("BipedalWalker-v3"))
observation = env.reset()
fitness = 0

while True:
    env.render()
    
    input = torch.from_numpy(observation).float()
    action = best_agent.act(input).detach().numpy()
        
    observation, reward, done, info = env.step(action) 
    fitness += reward

    if done:
        break

env.close()
show_video()
print(fitness)

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


6.202955795563166
